In [9]:
import torch
import torch_pruning as tp
from yolox.models import create_yolox_model

model = create_yolox_model(num_classes=20,name = 'yolox_custom',  
exp_path = '/workspace/tensorrt/YOLOX/exps/default/custom2/type3/backbone50_all25_stem/yolox_voc_tiny_backbone50_all25_lr5.py',
ckpt_path = '/workspace/tensorrt/YOLOX/fpgm_2/type3/second/yolox_voc_tiny_backbone50_all25_lr5/37.12.pth'
).cuda()
# 1. setup strategy (L1 Norm)
strategy = tp.strategy.L1Strategy() # or tp.strategy.RandomStrategy()

In [10]:

# 2. build dependency graph for resnet18
DG = tp.DependencyGraph()
DG.build_dependency(model, example_inputs=torch.randn(1,3,224,224).cuda())



In [11]:

temp = []
temp2 = list(model.state_dict().keys())
num_list = [0,1,2,3]
#print(temp2)
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('2.0','2[0]')
            temp2[i] = temp2[i].replace('2.1','2[1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp.append(temp2[i])

print(temp)
# backbone 만 프루닝


['backbone.backbone.stem.conv.conv', 'backbone.backbone.dark2[0].conv', 'backbone.backbone.dark2[1].conv1.conv', 'backbone.backbone.dark2[1].conv2.conv', 'backbone.backbone.dark2[1].conv3.conv', 'backbone.backbone.dark2[1].m[0].conv1.conv', 'backbone.backbone.dark2[1].m[0].conv2.conv', 'backbone.backbone.dark3[0].conv', 'backbone.backbone.dark3[1].conv1.conv', 'backbone.backbone.dark3[1].conv2.conv', 'backbone.backbone.dark3[1].conv3.conv', 'backbone.backbone.dark3[1].m[0].conv1.conv', 'backbone.backbone.dark3[1].m[0].conv2.conv', 'backbone.backbone.dark3[1].m[1].conv1.conv', 'backbone.backbone.dark3[1].m[1].conv2.conv', 'backbone.backbone.dark3[1].m[2].conv1.conv', 'backbone.backbone.dark3[1].m[2].conv2.conv', 'backbone.backbone.dark4[0].conv', 'backbone.backbone.dark4[1].conv1.conv', 'backbone.backbone.dark4[1].conv2.conv', 'backbone.backbone.dark4[1].conv3.conv', 'backbone.backbone.dark4[1].m[0].conv1.conv', 'backbone.backbone.dark4[1].m[0].conv2.conv', 'backbone.backbone.dark4[1].m

In [12]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [13]:
from pruning_imagenet import Mask

In [14]:
m = Mask(model)
#m.mask_index = [24,27,33,39,45,54,57,63,69,75,90,93,99,102, ] # backbone conv3 30 60 96
                           # ,105,108,111,114,117,120,123,141,159] # neck

m.mask_index = [30 ,60 ,96 ,105,108,111,114,117,120,123,141,159,126,129,132,135,138,144,147,150,153,156,162,165,168,171,174, #neck
                            231,234,237, #stems
                            177,180,183,186,189,192,195,198,201,204,207,210] # head

        

In [15]:
m.init_length()

In [16]:
m.init_mask(1, 1/3)
print(m.mask_index)

filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done


In [17]:
m.do_mask()
m.do_similar_mask()

mask Done
mask similar Done


In [18]:
m.if_zero()
print(m.mask_index)
print(m.similar_matrix)

layer: 30, number of nonzero weight is 2304, zero is 1152
layer: 60, number of nonzero weight is 9216, zero is 4608
layer: 96, number of nonzero weight is 36864, zero is 18432
layer: 105, number of nonzero weight is 27648, zero is 13824
layer: 108, number of nonzero weight is 13824, zero is 6912
layer: 111, number of nonzero weight is 13824, zero is 6912
layer: 114, number of nonzero weight is 13824, zero is 6912
layer: 117, number of nonzero weight is 3456, zero is 1728
layer: 120, number of nonzero weight is 31104, zero is 15552
layer: 123, number of nonzero weight is 6912, zero is 3456
layer: 126, number of nonzero weight is 3456, zero is 1728
layer: 129, number of nonzero weight is 3456, zero is 1728
layer: 132, number of nonzero weight is 3456, zero is 1728
layer: 135, number of nonzero weight is 864, zero is 432
layer: 138, number of nonzero weight is 7776, zero is 3888
layer: 141, number of nonzero weight is 31104, zero is 15552
layer: 144, number of nonzero weight is 6912, zero

In [19]:
model = m.model


In [20]:
dict ={}


for key, value in model.state_dict().items():
    temp = []
    if len(value.shape) ==4:
        print('pruning channel and layer name : ',len(value),',', key)
        for i in range(len(value)):
            if value[i].sum() == 0: 
                print('pruning channex index : ', i)
                #temp = torch.norm(value.view(len(value), -1) , p= 1, dim=1)
                #print(temp)
                #dict[key].append(i)
                temp.append(i)
    if temp:   
        dict[key]  = temp

pruning channel and layer name :  24 , backbone.backbone.stem.conv.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.0.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv2.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.1.conv3.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv2.conv.weight
pruning channel and layer name :  96 , backbone.backbone.dark3.0.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark3.1.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark3.1.conv2.conv.weight
pruning channel and layer name :  72 , backbone.backbone.dark3.1.conv3.conv.weight
pruning channex index :  2
pruning channex index :  3
pruning channex index :  6
pruning channe

In [21]:
print(dict)
#del dict['backbone.backbone.dark3.1.conv3.conv.weight'][0]

{'backbone.backbone.dark3.1.conv3.conv.weight': [2, 3, 6, 7, 15, 16, 22, 33, 34, 36, 38, 42, 44, 47, 50, 51, 52, 53, 57, 59, 60, 63, 70, 71], 'backbone.backbone.dark4.1.conv3.conv.weight': [1, 3, 5, 12, 13, 14, 17, 19, 21, 29, 30, 31, 33, 34, 37, 40, 41, 43, 46, 48, 49, 51, 59, 60, 66, 68, 72, 73, 75, 76, 90, 92, 97, 99, 100, 103, 105, 111, 113, 114, 117, 122, 125, 130, 131, 132, 136, 142], 'backbone.backbone.dark5.2.conv3.conv.weight': [3, 5, 10, 17, 18, 27, 32, 37, 41, 44, 47, 49, 50, 55, 56, 58, 62, 65, 66, 72, 76, 78, 80, 82, 84, 86, 88, 94, 97, 101, 102, 105, 107, 109, 110, 111, 115, 116, 117, 124, 127, 129, 132, 134, 140, 146, 153, 157, 160, 162, 165, 166, 168, 173, 175, 176, 178, 181, 190, 191, 195, 196, 198, 201, 205, 207, 210, 212, 213, 214, 215, 218, 219, 221, 225, 227, 232, 235, 238, 240, 241, 242, 246, 250, 254, 258, 261, 265, 266, 269, 270, 271, 276, 278, 286, 287], 'backbone.lateral_conv0.conv.weight': [3, 5, 7, 8, 12, 21, 24, 25, 28, 30, 31, 32, 36, 39, 42, 44, 47, 49, 5

In [22]:

temp = []
temp2 = list(dict.keys())
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('.0.0','[0][0]')
            temp2[i] = temp2[i].replace('.0.1','[0][1]')
            temp2[i] = temp2[i].replace('.1.0','[1][0]')
            temp2[i] = temp2[i].replace('.1.1','[1][1]')
            temp2[i] = temp2[i].replace('.2.0','[2][0]')
            temp2[i] = temp2[i].replace('.2.1','[2][1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp2[i] = temp2[i].replace('.0','[0]')
            temp2[i] = temp2[i].replace('.1','[1]')
            temp2[i] = temp2[i].replace('.2','[2]')
            temp.append(temp2[i])

#temp.insert(27,'head.stems[0].conv')
#temp.insert(28,'head.stems[1].conv')
#temp.insert(29,'head.stems[2].conv')

del temp[-1]
del temp[-1]
del temp[-1]

temp.remove('head.cls_convs[2][1].conv')
temp.remove('head.reg_convs[2][1].conv')
temp.remove('head.reg_convs[1][1].conv')
temp.remove('head.cls_convs[1][1].conv')
temp.remove('head.reg_convs[0][1].conv')
temp.remove('head.cls_convs[0][1].conv')
#temp.remove('head.cls_convs[1][0].conv')
#temp.remove('head.reg_convs[1][0].conv')
import copy
print(temp)
key = []
temp2 = copy.deepcopy(temp)
for i in range(len(temp2)):
      
            temp2[i] = temp2[i].replace('[0][0]','.0.0')
            temp2[i] = temp2[i].replace('[0][1]','.0.1',)
            temp2[i] = temp2[i].replace('[1][0]','.1.0',)
            temp2[i] = temp2[i].replace('[1][1]','.1.1',)
            temp2[i] = temp2[i].replace('[2][0]','.2.0',)
            temp2[i] = temp2[i].replace('[2][1]','.2.1',)
            temp2[i] = temp2[i].replace('3[0]','3.0',)
            temp2[i] = temp2[i].replace('3[1]','3.1')
            temp2[i] = temp2[i].replace('4[0]','4.0',)
            temp2[i] = temp2[i].replace('4[1]','4.1',)
            temp2[i] = temp2[i].replace('5[0]','5.0',)
            temp2[i] = temp2[i].replace('5[1]','5.1',)
            temp2[i] = temp2[i].replace('5[2]','5.2',)
            temp2[i] = temp2[i].replace('m[0]','m.0',)
            temp2[i] = temp2[i].replace('m[1]','m.1',)
            temp2[i] = temp2[i].replace('m[2]','m.2',)
            temp2[i] = temp2[i].replace('[0]','.0',)
            temp2[i] = temp2[i].replace('[1]','.1',)
            temp2[i] = temp2[i].replace('[2]','.2',)
            key.append(temp2[i])

print(key)
print(temp)

['backbone.backbone.dark3[1].conv3.conv', 'backbone.backbone.dark4[1].conv3.conv', 'backbone.backbone.dark5[2].conv3.conv', 'backbone.lateral_conv0.conv', 'backbone.C3_p4.conv1.conv', 'backbone.C3_p4.conv2.conv', 'backbone.C3_p4.conv3.conv', 'backbone.C3_p4.m[0].conv1.conv', 'backbone.C3_p4.m[0].conv2.conv', 'backbone.reduce_conv1.conv', 'backbone.C3_p3.conv1.conv', 'backbone.C3_p3.conv2.conv', 'backbone.C3_p3.conv3.conv', 'backbone.C3_p3.m[0].conv1.conv', 'backbone.C3_p3.m[0].conv2.conv', 'backbone.bu_conv2.conv', 'backbone.C3_n3.conv1.conv', 'backbone.C3_n3.conv2.conv', 'backbone.C3_n3.conv3.conv', 'backbone.C3_n3.m[0].conv1.conv', 'backbone.C3_n3.m[0].conv2.conv', 'backbone.bu_conv1.conv', 'backbone.C3_n4.conv1.conv', 'backbone.C3_n4.conv2.conv', 'backbone.C3_n4.conv3.conv', 'backbone.C3_n4.m[0].conv1.conv', 'backbone.C3_n4.m[0].conv2.conv', 'head.cls_convs[0][0].conv', 'head.cls_convs[1][0].conv', 'head.cls_convs[2][0].conv', 'head.reg_convs[0][0].conv', 'head.reg_convs[1][0].conv'

In [23]:
value = list(dict.values())
value

[[2,
  3,
  6,
  7,
  15,
  16,
  22,
  33,
  34,
  36,
  38,
  42,
  44,
  47,
  50,
  51,
  52,
  53,
  57,
  59,
  60,
  63,
  70,
  71],
 [1,
  3,
  5,
  12,
  13,
  14,
  17,
  19,
  21,
  29,
  30,
  31,
  33,
  34,
  37,
  40,
  41,
  43,
  46,
  48,
  49,
  51,
  59,
  60,
  66,
  68,
  72,
  73,
  75,
  76,
  90,
  92,
  97,
  99,
  100,
  103,
  105,
  111,
  113,
  114,
  117,
  122,
  125,
  130,
  131,
  132,
  136,
  142],
 [3,
  5,
  10,
  17,
  18,
  27,
  32,
  37,
  41,
  44,
  47,
  49,
  50,
  55,
  56,
  58,
  62,
  65,
  66,
  72,
  76,
  78,
  80,
  82,
  84,
  86,
  88,
  94,
  97,
  101,
  102,
  105,
  107,
  109,
  110,
  111,
  115,
  116,
  117,
  124,
  127,
  129,
  132,
  134,
  140,
  146,
  153,
  157,
  160,
  162,
  165,
  166,
  168,
  173,
  175,
  176,
  178,
  181,
  190,
  191,
  195,
  196,
  198,
  201,
  205,
  207,
  210,
  212,
  213,
  214,
  215,
  218,
  219,
  221,
  225,
  227,
  232,
  235,
  238,
  240,
  241,
  242,
  246,
  250,
  

In [24]:
for i in range(len(temp)):
    #pruning_plan = DG.get_pruning_plan(eval(f"model.{temp[i]}"), tp.prune_conv_out_channel, value[i])
    pruning_plan = DG.get_pruning_plan(eval(f"model.{temp[i]}"), tp.prune_conv_out_channel, dict[(key[i]) + '.weight'])
    print(pruning_plan)
    if DG.check_pruning_plan(pruning_plan):
        pruning_plan.exec()     

Conv2d(48, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)

--------------------------------
          Pruning Plan
--------------------------------
User pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv3.conv (Conv2d(48, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)) => ConvOutChannelPruner on backbone.backbone.dark3.1.conv3.conv (Conv2d(48, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)), Index=[2, 3, 6, 7, 15, 16, 22, 33, 34, 36, 38, 42, 44, 47, 50, 51, 52, 53, 57, 59, 60, 63, 70, 71], metric={'#params': 1152}]

Coupled pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv3.conv (Conv2d(48, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)) => BatchnormPruner on backbone.backbone.dark3.1.conv3.bn (BatchNorm2d(72, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)), Index=[2, 3, 6, 7, 15, 16, 22, 33, 34, 36, 38, 42, 44, 47, 50, 51, 52, 53, 57, 59, 60, 63, 70, 71], metric={'#params': 48}]
[ [DEP] BatchnormPruner on bac

In [25]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [26]:
ckpt_state = {
                "model": model.state_dict(),
            }

torch.save(ckpt_state, 'type3_0.50_a_v2.pth')


